In [1]:
# FOLLOW WHALE HOMEWORK FOR ANALYSIS GUIDANCE

In [1]:
# IMPORT LIBARIES AND DEPENDENCIES 
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from pathlib import Path
from dotenv import load_dotenv
%matplotlib inline
import seaborn as sns

In [2]:
# PULL IN SECTOR / STOCK DATA FOR 2020
#IMPORT DATA FOLLOWING P1_Q3 FOR THREE SECTORS/STOCKS
#Walmart, P&G, J&J. \\\
load_dotenv()

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# api = tradeapi.REST()
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [3]:
# CLEANSE AND NORMALIZE DATA
# COUNT AND DROP NULLS
list_of_tickers = ["WMT", "JNJ", "PG","JPM", "MCD","XOM"]

# YOUR CODE HERE
# Get all Asstes
assets = api.list_assets()

In [4]:
#COMBINE DATA INTO ONE DF
# api = tradeapi.REST()
api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

list_of_tickers = ["WMT", "JNJ", "PG","JPM", "MCD","XOM"]

# YOUR CODE HERE
# Get all Asstes
assets = api.list_assets()

# Keep only tradeable assets
tradeable = [asset for asset in assets if asset.tradable ]

In [5]:
# Create a new empty DataFrame
asset_info_df = pd.DataFrame()
asset_info_df['symbol'] = pd.Series([asset.symbol for asset in assets])

# Display the first 10 asset tickers
asset_info_df.sample(5)

,symbol
5428,SCX
1743,IR
4863,CUBE
5179,RIGS
6710,WEYS


In [13]:
resil_tickers = ["WMT", "JNJ", "PG"]

# Set timeframe to '1D'
timeframe = '1D'

# Set start and end datetimes of 1 year, between now and 365 days ago.
end_date = datetime.now()
start_date = end_date + timedelta(-88)

# Get 1 year's worth of historical data for WMT, JNJ and PG
dfr = api.get_barset(
    resil_tickers,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

dfr.head(10)

JNJ                                        \
                             open      high       low   close    volume   
2019-12-18 00:00:00-05:00  143.72  143.8800  142.7400  143.18   3978012   
2019-12-19 00:00:00-05:00  143.89  145.8800  143.6100  145.35   6053480   
2019-12-20 00:00:00-05:00  146.88  147.8400  144.8000  146.07  10594866   
2019-12-23 00:00:00-05:00  145.74  146.7600  145.3401  146.41   3575567   
2019-12-24 00:00:00-05:00  146.00  146.0100  145.2600  145.97   1729932   
2019-12-26 00:00:00-05:00  145.97  146.1827  145.1600  145.83   2233533   
2019-12-27 00:00:00-05:00  146.20  146.5500  145.5400  145.76   1939538   
2019-12-30 00:00:00-05:00  145.78  146.2000  145.0300  145.35   2089996   
2019-12-31 00:00:00-05:00  145.10  146.0100  144.7800  145.82   2541672   
2020-01-02 00:00:00-05:00  145.87  146.0200  145.0800  146.00   4380284   

                               PG                                       \
                             open      high      low   close    volume   
2019-12-18 00:00:00-05:00  125.31  125.8700  123.690  124.01   5119223   
2019-12-19 00:00:00-05:00  124.16  125.2000  123.900  124.91   4262123   
2019-12-20 00:00:00-05:00  126.15  126.6000  125.030  125.37  10570598   
2019-12-23 00:00:00-05:00  125.52  125.8200  124.685  124.91   4649046   
2019-12-24 00:00:00-05:00  124.79  125.3099  124.580  125.18   1798951   
2019-12-26 00:00:00-05:00  125.10  125.7450  124.870  125.23   3566876   
2019-12-27 00:00:00-05:00  125.56  126.2950  125.390  126.07   2975061   
2019-12-30 00:00:00-05:00  125.76  125.9800  124.140  124.51   2931062   
2019-12-31 00:00:00-05:00  124.43  124.9900  123.950  124.87   2986857   
2020-01-02 00:00:00-05:00  124.50  124.7300  122.940  123.41   5965498   

                              WMT                                      
                             open     high       low   close   volume  
2019-12-18 00:00:00-05:00  121.51  121.680  119.8400  119.87  3583580  
2019-12-19 00:00:00-05:00  119.94  120.505  119.7800  120.05  2882602  
2019-12-20 00:00:00-05:00  121.48  122.120  120.0200  120.29  6628305  
2019-12-23 00:00:00-05:00  120.20  120.690  119.0100  119.03  3504298  
2019-12-24 00:00:00-05:00  119.13  119.600  119.0100  119.52  1991969  
2019-12-26 00:00:00-05:00  119.61  119.850  118.9500  119.55  3062390  
2019-12-27 00:00:00-05:00  119.94  119.940  119.2450  119.57  2557506  
2019-12-30 00:00:00-05:00  119.80  119.810  119.1300  119.42  2373893  
2019-12-31 00:00:00-05:00  119.50  119.800  118.1900  118.79  3439010  
2020-01-02 00:00:00-05:00  118.86  119.890  118.7016  118.94  5449703

In [3]:
nonresil_tickers = ["JPM", "MCD", "XOM"]

# Set timeframe to '1D'
timeframe = '1D'

# Set start and end datetimes of 1 year, between now and 365 days ago.
end_date = datetime.now()
start_date = end_date + timedelta(-88)

# Get 1 year's worth of historical data for JPM, MCD and XOM
dfnr = api.get_barset(
    nonresil_tickers,
    timeframe,
    limit=None,
    start=start_date,
    end=end_date,
    after=None,
    until=None,
).df

dfnr.head(10)

JPM                                         MCD  \
                             open    high       low   close    volume    open   
2019-12-18 00:00:00-05:00  138.15  139.02  137.7151  138.07   7422871  196.60   
2019-12-19 00:00:00-05:00  137.76  138.22  137.1000  137.35   6637532  195.46   
2019-12-20 00:00:00-05:00  138.02  138.88  136.9000  137.24  16546366  198.55   
2019-12-23 00:00:00-05:00  137.60  137.91  136.8800  137.22   6193775  197.54   
2019-12-24 00:00:00-05:00  137.33  137.63  137.0641  137.58   2824296  195.93   
2019-12-26 00:00:00-05:00  137.89  139.10  137.7800  139.09   4753888  196.61   
2019-12-27 00:00:00-05:00  139.30  139.77  138.6697  139.14   4864945  197.33   
2019-12-30 00:00:00-05:00  139.90  140.08  138.4700  138.68   5571805  197.58   
2019-12-31 00:00:00-05:00  138.51  139.48  138.2950  139.40   4896689  196.80   
2020-01-02 00:00:00-05:00  139.79  141.10  139.2650  141.09   8484783  198.00   

                                                                 XOM           \
                              high       low   close   volume   open     high   
2019-12-18 00:00:00-05:00  196.790  194.7101  195.75  2795111  69.63  70.2900   
2019-12-19 00:00:00-05:00  197.175  195.2500  197.05  2042418  69.99  70.0500   
2019-12-20 00:00:00-05:00  199.000  196.6500  197.16  3874156  69.86  70.2267   
2019-12-23 00:00:00-05:00  197.540  196.0300  196.23  1675904  69.86  70.3000   
2019-12-24 00:00:00-05:00  197.140  195.6700  196.68   712857  70.35  70.5000   
2019-12-26 00:00:00-05:00  197.160  196.1627  197.16  1212708  70.19  70.5000   
2019-12-27 00:00:00-05:00  198.470  197.1750  198.16  1504372  70.20  70.3123   
2019-12-30 00:00:00-05:00  197.800  195.9400  196.99  1780512  70.09  70.4350   
2019-12-31 00:00:00-05:00  197.950  196.5579  197.55  1694891  69.02  69.8000   
2020-01-02 00:00:00-05:00  200.800  197.8100  200.80  2886391  70.24  71.0200   

                                                      
                               low   close    volume  
2019-12-18 00:00:00-05:00  69.2800  69.875   9061331  
2019-12-19 00:00:00-05:00  69.2400  69.380   8474644  
2019-12-20 00:00:00-05:00  69.3900  69.940  22033157  
2019-12-23 00:00:00-05:00  69.5300  70.290   9451860  
2019-12-24 00:00:00-05:00  69.9101  70.020   3650368  
2019-12-26 00:00:00-05:00  70.0100  70.130   6831328  
2019-12-27 00:00:00-05:00  69.8800  69.890   7459006  
2019-12-30 00:00:00-05:00  69.4000  69.480  10463554  
2019-12-31 00:00:00-05:00  69.0100  69.760   9526602  
2020-01-02 00:00:00-05:00  70.2368  70.900  10586614

In [8]:
#PRINT CONCAT DF

In [9]:
#PLOT DAILY RETURNS

In [7]:
#PLOT CUMULATIVE RETURNS

### Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the `ewm` with a 21 trading day (approximately one calendar month) half-life.

In [9]:
#CALCULATE RISK

In [10]:
#CALCULATE ROLLING 

In [ ]:
#CALCULATE SHARPE RATIOS